In [256]:
import pandas as pd
import app_functions.data_manipulation as dm
import app_functions.requests as req

config = dm.read_config()

# read sport as sport field in config
sport = config["sports"]
token = config["api_token"]
bookmakers = config["bookmakers"]

# get odds for each sport
all_odds = [] 
for s in sport:
    odds = req.get_odds_request(token, s, bookmakers)
    all_odds.append(odds)

# convert to dataframe
df = dm.json_to_df(all_odds)

In [257]:
import itertools

new_df = pd.DataFrame(columns=['game_id', 'bookie1', 'bookie2'])

for game_id, group in df.groupby('game_id'):
    bookies = group['bookmaker'].unique()
    for bookie1, bookie2 in itertools.combinations(bookies, 2):
        new_index = len(new_df)
        new_row = {'game_id': game_id, 'bookie1': bookie1, 'bookie2': bookie2}
        new_df.loc[new_index] = new_row

In [258]:
new_df.drop_duplicates(inplace=True)
new_df.dropna(inplace=True)
new_df.reset_index(drop=True, inplace=True)

In [259]:
df.head()

,sport,teams,bookmaker,bet type,event date,favorite,underdog,odds favorite,odds underdog,game_id
0,NFL,Baltimore Ravens_Kansas City Chiefs,draftkings,h2h,2024-09-06,Kansas City Chiefs,Baltimore Ravens,-155,130,Baltimore Ravens_Kansas City Chiefs_2024-09-06...
1,NFL,Baltimore Ravens_Kansas City Chiefs,fanduel,h2h,2024-09-06,Kansas City Chiefs,Baltimore Ravens,-144,122,Baltimore Ravens_Kansas City Chiefs_2024-09-06...
2,NFL,Baltimore Ravens_Kansas City Chiefs,betmgm,h2h,2024-09-06,Kansas City Chiefs,Baltimore Ravens,-150,125,Baltimore Ravens_Kansas City Chiefs_2024-09-06...
3,NFL,Green Bay Packers_Philadelphia Eagles,draftkings,h2h,2024-09-07,Philadelphia Eagles,Green Bay Packers,-125,105,Green Bay Packers_Philadelphia Eagles_2024-09-...
4,NFL,Green Bay Packers_Philadelphia Eagles,fanduel,h2h,2024-09-07,Philadelphia Eagles,Green Bay Packers,-120,102,Green Bay Packers_Philadelphia Eagles_2024-09-...


In [260]:
def get_bookie_team(row, bookie_num, type):
    # Build the column name dynamically based on bookie number and type
    odds_column = "favorite" if type == "favorite" else "underdog"
    
    filtered_df = df[(df["bookmaker"] == row[bookie_num]) & (df["game_id"] == row["game_id"])]

    # Assuming there's only one matching row, get the specified odds value
    if not filtered_df.empty:
        return filtered_df[odds_column].iloc[0]
    else:
        return None  # or some default value
    
def get_odds(row, bookie_num, type):
    odds_column = "odds favorite" if type == "favorite" else "odds underdog"
    
    # Filter df for the current row's bookie and game_id
    filtered_df = df[(df["bookmaker"] == row[bookie_num]) & (df["game_id"] == row["game_id"])]
    
    # Assuming there's only one matching row, get the specified odds value
    if not filtered_df.empty:
        return filtered_df[odds_column].iloc[0]
    else:
        return None  # or some default value

In [261]:
new_df["bookie1_favorite_odds"] = new_df.apply(get_odds, bookie_num='bookie1', type='favorite', axis=1)
new_df["bookie1_favorite"] = new_df.apply(get_bookie_team, bookie_num = 'bookie1', type='favorite', axis=1)
new_df["bookie2_favorite_odds"] = new_df.apply(get_odds, bookie_num='bookie2', type='favorite', axis=1)
new_df["bookie2_favorite"] = new_df.apply(get_bookie_team, bookie_num = 'bookie2', type='favorite', axis=1)
new_df["bookie1_underdog_odds"] = new_df.apply(get_odds, bookie_num='bookie1', type='underdog', axis=1)
new_df["bookie1_underdog"] = new_df.apply(get_bookie_team, bookie_num = 'bookie1', type='underdog', axis=1)
new_df["bookie2_underdog_odds"] = new_df.apply(get_odds, bookie_num='bookie2', type='underdog', axis=1)
new_df["bookie2_underdog"] = new_df.apply(get_bookie_team, bookie_num = 'bookie2', type='underdog', axis=1)


In [262]:
new_df.head()

,game_id,bookie1,bookie2,bookie1_favorite_odds,bookie1_favorite,bookie2_favorite_odds,bookie2_favorite,bookie1_underdog_odds,bookie1_underdog,bookie2_underdog_odds,bookie2_underdog
0,Arizona Cardinals_Buffalo Bills_2024-09-08_h2h,fanduel,draftkings,-330,Buffalo Bills,-355,Buffalo Bills,265,Arizona Cardinals,280,Arizona Cardinals
1,Arizona Cardinals_Buffalo Bills_2024-09-08_h2h,fanduel,betmgm,-330,Buffalo Bills,-350,Buffalo Bills,265,Arizona Cardinals,260,Arizona Cardinals
2,Arizona Cardinals_Buffalo Bills_2024-09-08_h2h,draftkings,betmgm,-355,Buffalo Bills,-350,Buffalo Bills,280,Arizona Cardinals,260,Arizona Cardinals
3,Atlanta Falcons_Philadelphia Eagles_2024-09-17...,fanduel,draftkings,-176,Philadelphia Eagles,-166,Philadelphia Eagles,148,Atlanta Falcons,140,Atlanta Falcons
4,Atlanta Falcons_Pittsburgh Steelers_2024-09-08...,draftkings,fanduel,-142,Atlanta Falcons,-142,Atlanta Falcons,120,Pittsburgh Steelers,120,Pittsburgh Steelers


In [263]:
new_df = new_df.dropna()

In [264]:
new_df["max_favorite_odds"] = new_df[["bookie1_favorite_odds", "bookie2_favorite_odds"]].max(axis=1)
new_df["max_underdog_odds"] = new_df[["bookie1_underdog_odds", "bookie2_underdog_odds"]].max(axis=1)
new_df["favorite_diff"] = (new_df["bookie1_favorite_odds"] - new_df["bookie2_favorite_odds"]).abs()

In [265]:
# create new df with only the rows with the max difference in odds for each game_id
diff_df = new_df.loc[new_df.groupby('game_id')['favorite_diff'].idxmax()]
diff_df.dropna(inplace=True)
diff_df.reset_index(drop=True, inplace=True)

In [266]:
diff_df.head()

,game_id,bookie1,bookie2,bookie1_favorite_odds,bookie1_favorite,bookie2_favorite_odds,bookie2_favorite,bookie1_underdog_odds,bookie1_underdog,bookie2_underdog_odds,bookie2_underdog,max_favorite_odds,max_underdog_odds,favorite_diff
0,Arizona Cardinals_Buffalo Bills_2024-09-08_h2h,fanduel,draftkings,-330,Buffalo Bills,-355,Buffalo Bills,265,Arizona Cardinals,280,Arizona Cardinals,-330,280,25
1,Atlanta Falcons_Philadelphia Eagles_2024-09-17...,fanduel,draftkings,-176,Philadelphia Eagles,-166,Philadelphia Eagles,148,Atlanta Falcons,140,Atlanta Falcons,-166,148,10
2,Atlanta Falcons_Pittsburgh Steelers_2024-09-08...,draftkings,betmgm,-142,Atlanta Falcons,-145,Atlanta Falcons,120,Pittsburgh Steelers,120,Pittsburgh Steelers,-142,120,3
3,Baltimore Ravens_Houston Texans_2024-12-25_h2h,fanduel,betmgm,-126,Baltimore Ravens,-120,Baltimore Ravens,108,Houston Texans,100,Houston Texans,-120,108,6
4,Baltimore Ravens_Kansas City Chiefs_2024-09-06...,draftkings,fanduel,-155,Kansas City Chiefs,-144,Kansas City Chiefs,130,Baltimore Ravens,122,Baltimore Ravens,-144,130,11


In [267]:
diff_df["favorite_bookie"] = diff_df.apply(lambda row: row["bookie1"] if row["bookie1_favorite_odds"] > row["bookie2_favorite_odds"] else row["bookie2"], axis=1)
diff_df["underdog_bookie"] = diff_df.apply(lambda row: row["bookie1"] if row["bookie1_underdog_odds"] > row["bookie2_underdog_odds"] else row["bookie2"], axis=1)

diff_df["favorite_bookie_team"] = diff_df.apply(lambda row: row["bookie1_favorite"] if row["bookie1_favorite_odds"] > row["bookie2_favorite_odds"] else row["bookie2_favorite"], axis=1)
diff_df["underdog_bookie_team"] = diff_df.apply(lambda row: row["bookie1_underdog"] if row["bookie1_underdog_odds"] > row["bookie2_underdog_odds"] else row["bookie2_underdog"], axis=1)

In [268]:
diff_df.head()

,game_id,bookie1,bookie2,bookie1_favorite_odds,bookie1_favorite,bookie2_favorite_odds,bookie2_favorite,bookie1_underdog_odds,bookie1_underdog,bookie2_underdog_odds,bookie2_underdog,max_favorite_odds,max_underdog_odds,favorite_diff,favorite_bookie,underdog_bookie,favorite_bookie_team,underdog_bookie_team
0,Arizona Cardinals_Buffalo Bills_2024-09-08_h2h,fanduel,draftkings,-330,Buffalo Bills,-355,Buffalo Bills,265,Arizona Cardinals,280,Arizona Cardinals,-330,280,25,fanduel,draftkings,Buffalo Bills,Arizona Cardinals
1,Atlanta Falcons_Philadelphia Eagles_2024-09-17...,fanduel,draftkings,-176,Philadelphia Eagles,-166,Philadelphia Eagles,148,Atlanta Falcons,140,Atlanta Falcons,-166,148,10,draftkings,fanduel,Philadelphia Eagles,Atlanta Falcons
2,Atlanta Falcons_Pittsburgh Steelers_2024-09-08...,draftkings,betmgm,-142,Atlanta Falcons,-145,Atlanta Falcons,120,Pittsburgh Steelers,120,Pittsburgh Steelers,-142,120,3,draftkings,betmgm,Atlanta Falcons,Pittsburgh Steelers
3,Baltimore Ravens_Houston Texans_2024-12-25_h2h,fanduel,betmgm,-126,Baltimore Ravens,-120,Baltimore Ravens,108,Houston Texans,100,Houston Texans,-120,108,6,betmgm,fanduel,Baltimore Ravens,Houston Texans
4,Baltimore Ravens_Kansas City Chiefs_2024-09-06...,draftkings,fanduel,-155,Kansas City Chiefs,-144,Kansas City Chiefs,130,Baltimore Ravens,122,Baltimore Ravens,-144,130,11,fanduel,draftkings,Kansas City Chiefs,Baltimore Ravens


In [269]:
needed_cols = ["game_id", "favorite_bookie", "favorite_bookie_team", "max_favorite_odds", "underdog_bookie", "underdog_bookie_team", "max_underdog_odds"]
arbitrage_df = diff_df[needed_cols]
arbitrage_df["arb_available"] = False

C:\Users\owurs\AppData\Local\Temp\ipykernel_18548\3514546116.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arbitrage_df["arb_available"] = False


In [270]:
arbitrage_df.head()

,game_id,favorite_bookie,favorite_bookie_team,max_favorite_odds,underdog_bookie,underdog_bookie_team,max_underdog_odds,arb_available
0,Arizona Cardinals_Buffalo Bills_2024-09-08_h2h,fanduel,Buffalo Bills,-330,draftkings,Arizona Cardinals,280,False
1,Atlanta Falcons_Philadelphia Eagles_2024-09-17...,draftkings,Philadelphia Eagles,-166,fanduel,Atlanta Falcons,148,False
2,Atlanta Falcons_Pittsburgh Steelers_2024-09-08...,draftkings,Atlanta Falcons,-142,betmgm,Pittsburgh Steelers,120,False
3,Baltimore Ravens_Houston Texans_2024-12-25_h2h,betmgm,Baltimore Ravens,-120,fanduel,Houston Texans,108,False
4,Baltimore Ravens_Kansas City Chiefs_2024-09-06...,fanduel,Kansas City Chiefs,-144,draftkings,Baltimore Ravens,130,False


In [271]:
# print rows where bookies have different favorite teams
# prints the obvious arbitrage opportunities (there are rarely any)
for index, row in diff_df.iterrows():
    if (row["bookie1_favorite"] != row["bookie2_favorite"] and (row["bookie1_favorite_odds"] > 100) and (row["bookie2_favorite_odds"] > 100)):
        print(row)

In [272]:
# add bet amount columns for when arb is available and arb estimated profit
arbitrage_df["favorite_bet_amount"] = 0
arbitrage_df["underdog_bet_amount"] = 0
arbitrage_df["arb_min_profit"] = 0

C:\Users\owurs\AppData\Local\Temp\ipykernel_18548\3080679581.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arbitrage_df["favorite_bet_amount"] = 0
C:\Users\owurs\AppData\Local\Temp\ipykernel_18548\3080679581.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arbitrage_df["underdog_bet_amount"] = 0
C:\Users\owurs\AppData\Local\Temp\ipykernel_18548\3080679581.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [273]:
def fav_bet_winnings(bet_amount, odds):
    return ((bet_amount * 100 / (odds)) + bet_amount)

def dog_bet_winnings(bet_amount, odds):
    return ((bet_amount * odds / 100) + bet_amount)

def bet_winnings(bet_amount, odds):
    if odds > 100:
        return fav_bet_winnings(bet_amount, odds)
    else:
        return dog_bet_winnings(bet_amount, odds)
    

In [274]:
def arb_available(dog_odds, dog_bet, fav_odds, fav_bet):
    if((bet_winnings(dog_bet, dog_odds) > dog_bet + fav_bet) and (bet_winnings(fav_bet, fav_odds) > dog_bet + fav_bet)):
        return True
        
    return False

In [275]:
def set_arb(row):
    dog_odds = row["max_underdog_odds"]
    fav_odds = row["max_favorite_odds"]

    if (fav_odds > 100 and dog_odds > 100):
        row["arb_available"] = True
        row["favorite_bet_amount"] = 50
        row["underdog_bet_amount"] = 50
        row["arb_min_profit"] = 0

    optimal_i = 0
    min_profit = 0
    can_arb = False

    for i in range(1, 99):
        fav_bet = i
        dog_bet = 100 - i

        profit = min(bet_winnings(dog_bet, dog_odds), bet_winnings(fav_bet, fav_odds)) - 100

        if(arb_available(dog_odds, dog_bet, fav_odds, fav_bet)):
            can_arb = True
            
        if profit > min_profit:
            min_profit = profit
            optimal_i = i
        
    
    row["arb_available"] = can_arb
    row["favorite_bet_amount"] = optimal_i
    row["underdog_bet_amount"] = 100 - optimal_i
    row["arb_min_profit"] = min_profit

    return row

In [276]:
# apply the set_arb function to each row in the arbitrage_df
arbitrage_df = arbitrage_df.apply(set_arb, axis=1)

In [277]:
# print the rows where an arbitrage opportunity is available
arbitrage_df[arbitrage_df["arb_available"] == True].head(5)

,game_id,favorite_bookie,favorite_bookie_team,max_favorite_odds,underdog_bookie,underdog_bookie_team,max_underdog_odds,arb_available,favorite_bet_amount,underdog_bet_amount,arb_min_profit


In [278]:
arbitrage_df.head()

,game_id,favorite_bookie,favorite_bookie_team,max_favorite_odds,underdog_bookie,underdog_bookie_team,max_underdog_odds,arb_available,favorite_bet_amount,underdog_bet_amount,arb_min_profit
0,Arizona Cardinals_Buffalo Bills_2024-09-08_h2h,fanduel,Buffalo Bills,-330,draftkings,Arizona Cardinals,280,False,0,100,0
1,Atlanta Falcons_Philadelphia Eagles_2024-09-17...,draftkings,Philadelphia Eagles,-166,fanduel,Atlanta Falcons,148,False,0,100,0
2,Atlanta Falcons_Pittsburgh Steelers_2024-09-08...,draftkings,Atlanta Falcons,-142,betmgm,Pittsburgh Steelers,120,False,0,100,0
3,Baltimore Ravens_Houston Texans_2024-12-25_h2h,betmgm,Baltimore Ravens,-120,fanduel,Houston Texans,108,False,0,100,0
4,Baltimore Ravens_Kansas City Chiefs_2024-09-06...,fanduel,Kansas City Chiefs,-144,draftkings,Baltimore Ravens,130,False,0,100,0


In [279]:
# bills bet 80 20
print(dog_bet_winnings(15, 280))
print(fav_bet_winnings(85, -330))

57.0
59.24242424242424
